In [ ]:
from functools import reduce

In [ ]:
# EDA asset dataframe

# Hypothesis: Five cases of nan values in the dataframe
# 1. The first rows of the series has nan values
c0 = [1, 1, 0, 0, 0]

# 2. The last rows of the series has nan values
c1 = [0, 0, 0, 1, 1]


# 3. The middle rows of the series has nan values
c2 = [0, 1, 1, 0, 0]

# 4. The first few and last few rows of the series has nan values
c3 = [1, 1, 0, 0, 1]

# 5. No nan values in the series
c4 = [0, 0, 0, 0, 0]

In [ ]:
def reduce_same(result, x):
    if result[-1] == x:
        return result
    else:
        result.append(x)
        return result

In [ ]:
reduce(reduce_same, c0, [-1])

[-1, 1, 0]

In [ ]:
def make_unique_case_df(df):
    unique_case_df = pd.DataFrame()
    cases = {
        (-1, 1, 0): 0, # 존재하지 않았다가 나타난 경우
        (-1, 0, 1): 1, # 존재했다가 사라진 경우
        (-1, 0, 1, 0): 2, # 존재했다가 사라지다가 다시 나타난 경우
        (-1, 1, 0, 1): 3, # 존재하지 않았다가 나타나다가 사라진 경우
        (-1, 1): 4, # 계속 존재하지 않은 경우
        (-1, 0): 5, # 계속 존재하는 경우
    }
    case_satisfied = [0] * len(cases.keys())
    result_cases = []

    for col in df.columns:
        series = df[col]
        isna = (series.isna() * 1).tolist()

        case_tuple = tuple(reduce(reduce_same, isna, [-1]))
        
        assert case_tuple in cases.keys() , f"Case tuple {case_tuple} not in cases"

        if case_satisfied[cases[case_tuple]] != 1:
            unique_case_df[col] = series
            case_satisfied[cases[case_tuple]] = 1
            result_cases.append(cases[case_tuple])
        else:
            if sum(case_satisfied) == len(cases.keys()):
                break
            else:
                continue
        
    return unique_case_df, result_cases

In [ ]:
uc_df, _ = make_unique_case_df(asset)

In [ ]:
uc_df

,AAPL
Date,
2018-01-02,169.714086
2018-01-03,169.684530
2018-01-04,170.472706
2018-01-05,172.413591
2018-01-08,171.773197
...,...
2018-11-26,174.620000
2018-11-27,174.240000
2018-11-28,180.940000


In [ ]:
def shorten_nan_cases(unique_case_df, minimum_rows_per_case=1):
    isna_df = unique_case_df.isna()

    unique_rows_df = isna_df.drop_duplicates()
    rowwise_cases = [tuple(c.values()) for c in list(unique_rows_df.T.to_dict().values())]
    cases_mapping = {case: i for i, case in enumerate(rowwise_cases)}

    case_satisfied_count = np.zeros(len(cases_mapping))

    if isinstance(minimum_rows_per_case, int):
        case_minimum_counts = np.ones(len(cases_mapping)) * minimum_rows_per_case
    elif isinstance(minimum_rows_per_case, list):
        assert len(minimum_rows_per_case) == len(cases_mapping), f"Required cases: {len(cases_mapping)} != Input cases: {len(minimum_rows_per_case)}"
        case_minimum_counts = minimum_rows_per_case
    
    boolmask_df = pd.DataFrame()
    
    already_sampled = [] # Assuming unique time-series index

    while not (case_satisfied_count >= case_minimum_counts).all():
        sample_row = isna_df.sample(1)

        if sample_row.index[0] in already_sampled:
            continue
        else:
            already_sampled.append(sample_row.index[0])
        
        row_case = tuple(sample_row.values[0])
        case_idx = cases_mapping[row_case]

        if case_satisfied_count[case_idx] >= case_minimum_counts[case_idx]:
            continue
        else:
            case_satisfied_count[case_idx] += 1
            boolmask_df = pd.concat([boolmask_df, sample_row], axis=0)
        
    result_df = unique_case_df.reindex(index=boolmask_df.index, columns=boolmask_df.columns).sort_index()

    return result_df


# 주의: 경계값을 생각하지 않음. 

In [ ]:
small_asset = shorten_nan_cases(uc_df, minimum_rows_per_case=[3, 3, 2, 1])
small_asset

,X005930,X302440,X326030,X011690,X322000
date,,,,,
2018-07-02,45550,NaN,NaN,16860.0,NaN
2018-07-13,46500,NaN,NaN,16780.0,NaN
2019-02-18,46200,NaN,NaN,17340.0,NaN
2020-01-13,60000,NaN,NaN,9990.0,18700.0
2020-03-10,54600,NaN,NaN,4180.0,23100.0
2020-03-17,47300,NaN,NaN,3720.0,20150.0
2020-07-15,54700,NaN,177000.0,2440.0,27300.0
2020-09-02,54400,NaN,181500.0,2440.0,28800.0
2020-09-04,55600,NaN,190000.0,NaN,NaN


In [ ]:
# small_asset = asset.iloc[:10, :5].copy()
# small_asset.iloc[:7, 0] = np.nan
# small_asset.iloc[3:5, 1] = np.nan
# small_asset.iloc[-7:, 2] = np.nan

# small_asset # 극단적인 케이스 가정하여 만듦

In [ ]:
small_asset_2d = small_asset.to_numpy()
small_asset_2d


array([[ 45550.,     nan,     nan,  16860.,     nan],
       [ 46500.,     nan,     nan,  16780.,     nan],
       [ 46200.,     nan,     nan,  17340.,     nan],
       [ 60000.,     nan,     nan,   9990.,  18700.],
       [ 54600.,     nan,     nan,   4180.,  23100.],
       [ 47300.,     nan,     nan,   3720.,  20150.],
       [ 54700.,     nan, 177000.,   2440.,  27300.],
       [ 54400.,     nan, 181500.,   2440.,  28800.],
       [ 55600.,     nan, 190000.,     nan,     nan]])

In [ ]:
small_asset_2d.shape

(9, 5)

In [ ]:
small_X = np.log(small_asset_2d[1:]/small_asset_2d[:-1])
small_X

array([[ 0.02064169,         nan,         nan, -0.00475625,         nan],
       [-0.00647251,         nan,         nan,  0.03282827,         nan],
       [ 0.26136476,         nan,         nan, -0.55143138,         nan],
       [-0.09431068,         nan,         nan, -0.87127335,  0.21130909],
       [-0.14352359,         nan,         nan, -0.11658758, -0.13662833],
       [ 0.14535341,         nan,         nan, -0.42172563,  0.30368241],
       [-0.00549956,         nan,  0.02510592,  0.        ,  0.05348868],
       [ 0.02181905,         nan,  0.04576842,         nan,         nan]])

In [ ]:
small_X.shape

(8, 5)

In [ ]:
# to address nan values, use pandas for covariance matrix

small_X_df = pd.DataFrame(small_X, columns=small_asset.columns)

In [ ]:
small_X_df.cov(min_periods=1)

,X005930,X302440,X326030,X011690,X322000
X005930,0.016420,NaN,0.000282,-0.009634,0.018035
X302440,NaN,NaN,NaN,NaN,NaN
X326030,0.000282,NaN,0.000213,NaN,NaN
X011690,-0.009634,NaN,NaN,0.120331,-0.048022
X322000,0.018035,NaN,NaN,-0.048022,0.037260


In [ ]:
small_Q = small_X_df.cov(min_periods=2).to_numpy()
small_Q

array([[ 0.01641976,         nan,  0.00028224, -0.00963374,  0.01803535],
       [        nan,         nan,         nan,         nan,         nan],
       [ 0.00028224,         nan,  0.00021347,         nan,         nan],
       [-0.00963374,         nan,         nan,  0.12033084, -0.04802211],
       [ 0.01803535,         nan,         nan, -0.04802211,  0.03725962]])

In [ ]:
np.isnan(small_Q).sum()

13

In [ ]:
small_Q.shape

(5, 5)

In [ ]:
small_r = np.nanmean(small_X, axis=0)
small_r = small_r.reshape(-1, 1)
small_r.shape


C:\Users\chlje\AppData\Local\Temp\ipykernel_44428\1298522824.py:1: RuntimeWarning: Mean of empty slice
  small_r = np.nanmean(small_X, axis=0)


(5, 1)

In [ ]:
small_r

array([[ 0.02492157],
       [        nan],
       [ 0.03543717],
       [-0.27613513],
       [ 0.10796297]])

In [ ]:
small_l = np.ones(small_r.shape)
small_l

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [ ]:
small_l.shape

(5, 1)

In [ ]:
small_Q_l_r = np.hstack([small_Q, small_l, small_r])
small_Q_l_r

array([[ 1.64197557e-02,             nan,  2.82235280e-04,
        -9.63373912e-03,  1.80353503e-02,  1.00000000e+00,
         2.49215722e-02],
       [            nan,             nan,             nan,
                    nan,             nan,  1.00000000e+00,
                    nan],
       [ 2.82235280e-04,             nan,  2.13469398e-04,
                    nan,             nan,  1.00000000e+00,
         3.54371698e-02],
       [-9.63373912e-03,             nan,             nan,
         1.20330837e-01, -4.80221091e-02,  1.00000000e+00,
        -2.76135130e-01],
       [ 1.80353503e-02,             nan,             nan,
        -4.80221091e-02,  3.72596244e-02,  1.00000000e+00,
         1.07962966e-01]])

In [ ]:
small_Q_l_r.shape

(5, 7)

In [ ]:
small_l_0_0 = np.hstack([small_l.T, [[0]], [[0]]])
small_l_0_0

array([[1., 1., 1., 1., 1., 0., 0.]])

In [ ]:
small_l_0_0.shape

(1, 7)

In [ ]:
small_r_0_0 = np.hstack([small_r.T, [[0]], [[0]]])
small_r_0_0

array([[ 0.02492157,         nan,  0.03543717, -0.27613513,  0.10796297,
         0.        ,  0.        ]])

In [ ]:
small_r_0_0.shape

(1, 7)

In [ ]:
small_L = np.vstack([small_Q_l_r, small_l_0_0, small_r_0_0])
small_L

array([[ 1.64197557e-02,             nan,  2.82235280e-04,
        -9.63373912e-03,  1.80353503e-02,  1.00000000e+00,
         2.49215722e-02],
       [            nan,             nan,             nan,
                    nan,             nan,  1.00000000e+00,
                    nan],
       [ 2.82235280e-04,             nan,  2.13469398e-04,
                    nan,             nan,  1.00000000e+00,
         3.54371698e-02],
       [-9.63373912e-03,             nan,             nan,
         1.20330837e-01, -4.80221091e-02,  1.00000000e+00,
        -2.76135130e-01],
       [ 1.80353503e-02,             nan,             nan,
        -4.80221091e-02,  3.72596244e-02,  1.00000000e+00,
         1.07962966e-01],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 2.49215722e-02,             nan,  3.54371698e-02,
        -2.76135130e-01,  1.07962966e-01,  0.00000000e+00,
         0.0000000

In [ ]:
small_L.shape

(7, 7)

In [ ]:
small_zero = np.zeros(small_l.shape)
small_zero

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [ ]:
small_zero_l_mu = np.vstack([small_zero, [[0]], [[MU]]])
small_zero_l_mu

array([[0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.01]])

In [ ]:
small_L_inv = np.linalg.inv(small_L)
small_L_inv.shape

(7, 7)

In [ ]:
small_L_inv

array([[nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan]])

In [ ]:
small_w_lmda1_lmda2 = small_L_inv @ small_zero_l_mu
small_w_lmda1_lmda2

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]])

In [ ]:
small_w = small_w_lmda1_lmda2[:-2]
small_lmda1 = small_w_lmda1_lmda2[-2]
small_lmda2 = small_w_lmda1_lmda2[-1]

In [ ]:
small_var = small_w.T @ small_Q @ small_w
small_var

array([[nan]])